In [1]:
import os
import pickle
import pandas as pd

In [2]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
df = pd.read_csv("EuroparlSentToPos.csv", encoding="UTF-8", sep=";")
df_filtered = df.dropna(subset=["pos"])
sentences = list(df_filtered["sent"])
all_pos = list(df_filtered["pos"])
df_filtered

,sent,pos
0,Genoptagelse af sessionen,NOUN ADP NOUN
1,"Jeg erklærer Europa-Parlamentets session, der ...",PRON VERB NOUN PUNCT PRON AUX VERB NOUN DET AD...
2,"Som De kan se, indfandt det store ""år 2000-pro...",ADP PRON AUX VERB VERB DET ADJ NOUN X PRON PUN...
3,De har udtrykt ønske om en debat om dette emne...,PRON AUX VERB NOUN ADP DET NOUN ADP DET NOUN A...
4,I mellemtiden ønsker jeg - som også en del kol...,ADP NOUN VERB PRON PUNCT ADP ADV DET NOUN NOUN...
...,...,...
1799995,"Jeg meddeler Dem, at jeg har modtaget et forsl...",PRON VERB PUNCT SCONJ PRON AUX VERB DET NOUN A...
1799996,Forhandlingen er afsluttet.,NOUN AUX VERB
1799997,Afstemningen finder sted i morgen kl. 12.00.,NOUN VERB NOUN ADP NOUN NOUN NUM
1799998,Tilsyn med forsikringsselskaber,NOUN ADP NOUN


In [3]:
def find_det_noun_pairs(lst):
    indexes = []
    for i in range(len(lst)):
        if lst[i] == "DET":
            for j in range(i+1, len(lst)):
                if lst[j] == "NOUN":
                    if all(x in ["ADV", "ADJ"] for x in lst[i+1:j]):
                        indexes.append((i, j))
                    break
    return indexes

In [4]:
testset = []
fælleskøn_labels = []
testset_sentences = []

for sent, pos in zip(sentences, all_pos):
    indexes = find_det_noun_pairs(pos.split())
    words = sent.split()
    for index in indexes:
        if words[index[0]][-1] == "n":
            fælleskøn_labels.append(True)
            testset.append(words[index[0]:index[1]+1])
            testset_sentences.append(sent)
        elif words[index[0]][-1] == "t":
            fælleskøn_labels.append(False)
            testset.append(words[index[0]:index[1]+1])
            testset_sentences.append(sent)


In [5]:
len(testset)

1535144

In [7]:
with open("GenderDict.pickle", "rb") as f:
    genderDict = pickle.load(f)

In [8]:
def get_stats(output=False):
    correct = 0
    total = 0
    no_guess = 0
    wrongs = []

    for set, label in zip(testset, fælleskøn_labels):
        if set[-1] in genderDict and set[0] in ["en", "et", "den", "det"]:
            total += 1
            if label == genderDict[set[-1]]:
                correct += 1
            else:
                wrongs.append((set, set[-1], genderDict[set[-1]]))
                #print(set, set[-1], genderDict[set[-1]])
        else:
            no_guess += 1
    if output: 
        print(f"Accuracy:  {round(correct/total*100, 2)} %")
        print(f"No guess: ", round(no_guess/len(testset)*100, 2), "%")
        print(f"Number of wrong: {len(wrongs)}")
    return len(wrongs), wrongs

nwrongs, wrongs = get_stats(output=True)

Accuracy:  99.81 %
No guess:  55.84 %
Number of wrong: 1287


In [9]:
genderDict["hus"]

False

In [11]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/DataProcessing/DanishDictionary/")
new_gender_dict = pickle.load(open("NewProducedDicts/GenderDict.pickle", "rb"))

In [14]:
len(new_gender_dict), len(genderDict)

(19848, 41575)